## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:  

[[total_active_users_per_day_full_table]]

### Table

In [5]:
query_text = """--sql
CREATE TABLE db1.total_active_users_per_day_rep_mobile_total
(
    `report_date` Date,
    `partner_uuid` String,
    `city` String,
    `total_active_users_per_day` Int32,
    `total_active_user_monetization_per_day` Int32,
    `total_active_user_subscribed_monetization_per_day` Int32
)
ENGINE = MergeTree()
ORDER BY partner_uuid
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [6]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.total_active_users_per_day_rep_mobile_total_mv
    REFRESH EVERY 1 DAY OFFSET 6 HOUR 5 MINUTE TO db1.total_active_users_per_day_rep_mobile_total AS
    SELECT
        report_date,
        partner_uuid,
        city,
        round(avg(citizen_id_count) OVER (PARTITION BY partner_uuid, city, toStartOfMonth(report_date)
            ORDER BY report_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        )) AS total_active_users_per_day,
        round(avg(total_active_user_monetization) OVER (PARTITION BY partner_uuid, city, toStartOfMonth(report_date)
            ORDER BY report_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        )) AS total_active_user_monetization_per_day,
        round(avg(total_active_user_subscribed_monetization) OVER (PARTITION BY partner_uuid, city, toStartOfMonth(report_date)
            ORDER BY report_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        )) AS total_active_user_subscribed_monetization_per_day
    FROM
        (SELECT
            report_date,
            partner_uuid,
            city,
            count(citizen_id) as citizen_id_count,
            count(if(monetization = 1,citizen_id,null)) as total_active_user_monetization,
            count(if(monetization = 1 and subscriptions_state = 'activated',citizen_id,null)) as total_active_user_subscribed_monetization
        FROM db1.total_active_users_per_day_full_table
        GROUP BY report_date,
                partner_uuid,
                city
        )
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [7]:
query_text = """--sql
    SELECT
        *
    FROM db1.total_active_users_per_day_rep_mobile_total
    WHERE city !=''
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)

report_date,partner_uuid,city,total_active_users_per_day,total_active_user_monetization_per_day,total_active_user_subscribed_monetization_per_day
date,str,str,i64,i64,i64
2024-08-27,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",2,0,0
2024-08-20,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",2,0,0
2024-08-19,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",2,0,0
2024-07-29,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",2,0,0
2024-07-28,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",2,0,0
…,…,…,…,…,…
2023-08-16,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",1,1,0
2023-08-12,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",1,1,0
2023-08-01,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""Казань""",1,1,0


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.total_activated_account_rep_mobile_full DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)

### drop mv

In [3]:
query_text = """--sql
    DROP TABLE db1.total_active_users_per_day_rep_mobile_total_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [4]:
query_text = """--sql
    DROP TABLE db1.total_active_users_per_day_rep_mobile_total
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.total_activated_account_rep_mobile_full_mv
"""

ch.query_run(query_text)